In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [3]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import gc
import seaborn as sns
import warnings
import scipy.signal as sg
warnings.filterwarnings("ignore")

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge

In [25]:
%%time
train = pd.read_csv('E:/kaggle/Molecular_properties/champs-scalar-coupling/train.csv')

Wall time: 5.14 s


In [56]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.807599999999994
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.257000000000000
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.254799999999999
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.254300000000001
4,4,dsgdb9nsd_000001,2,0,1JHC,84.807400000000001


In [ ]:
tmp = train[train['type'] == '2JHC']
tmp = tmp[tmp['atom_index_0'] == 3]
tmp = tmp[tmp['atom_index_1'] == 1]
tmp

In [26]:
%%time
test = pd.read_csv('E:/kaggle/Molecular_properties/champs-scalar-coupling/test.csv')

Wall time: 2.46 s


In [11]:
%%time
dipole_moments = pd.read_csv('E:/kaggle/Molecular_properties/champs-scalar-coupling/dipole_moments.csv')

Wall time: 545 ms


In [12]:
dipole_moments.shape

(85003, 4)

In [46]:
dipole_moments.head()

,molecule_name,X,Y,Z
0,dsgdb9nsd_000001,0.0000,0.0,0.0000
1,dsgdb9nsd_000002,-0.0002,0.0,1.6256
2,dsgdb9nsd_000003,0.0000,0.0,-1.8511
3,dsgdb9nsd_000005,0.0000,0.0,-2.8937
4,dsgdb9nsd_000007,0.0000,0.0,0.0000


In [48]:
%%time
mag_shield = pd.read_csv('E:/kaggle/Molecular_properties/champs-scalar-coupling/magnetic_shielding_tensors.csv')

Wall time: 4.27 s


In [49]:
mag_shield.shape

(1533537, 11)

In [50]:
mag_shield.head()

,molecule_name,atom_index,XX,YX,ZX,XY,YY,ZY,XZ,YZ,ZZ
0,dsgdb9nsd_000001,0,195.314999999999998,0.0000,-0.0001,0.0000,195.317000000000007,0.0007,-0.0001,0.0007,195.317000000000007
1,dsgdb9nsd_000001,1,31.340999999999998,-1.2317,4.0544,-1.2317,28.954599999999999,-1.7173,4.0546,-1.7173,34.086100000000002
2,dsgdb9nsd_000001,2,31.581399999999999,1.2173,-4.1474,1.2173,28.903600000000001,-1.6036,-4.1476,-1.6036,33.896700000000003
3,dsgdb9nsd_000001,3,31.517199999999999,4.1086,1.2723,4.1088,33.906799999999997,1.6950,1.2724,1.6951,28.957899999999999
4,dsgdb9nsd_000001,4,31.402899999999999,-4.0942,-1.1793,-4.0944,34.077599999999997,1.6259,-1.1795,1.6260,28.901299999999999


In [42]:
%%time
muli_charges = pd.read_csv('E:/kaggle/Molecular_properties/champs-scalar-coupling/mulliken_charges.csv')

Wall time: 1.48 s


In [51]:
muli_charges.shape

(1533537, 3)

In [52]:
muli_charges.head()

,molecule_name,atom_index,mulliken_charge
0,dsgdb9nsd_000001,0,-0.535689
1,dsgdb9nsd_000001,1,0.133921
2,dsgdb9nsd_000001,2,0.133922
3,dsgdb9nsd_000001,3,0.133923
4,dsgdb9nsd_000001,4,0.133923


In [58]:
%%time
potential_energy = pd.read_csv('E:/kaggle/Molecular_properties/champs-scalar-coupling/potential_energy.csv')

Wall time: 486 ms


In [59]:
potential_energy.shape

(85003, 2)

In [60]:
potential_energy.head()

,molecule_name,potential_energy
0,dsgdb9nsd_000001,-40.523679999999999
1,dsgdb9nsd_000002,-56.560250000000003
2,dsgdb9nsd_000003,-76.426079999999999
3,dsgdb9nsd_000005,-93.428489999999996
4,dsgdb9nsd_000007,-79.838690000000000


In [57]:
83.0224+0.2545+1.2586+0.2720

84.8075

In [61]:
%%time
scalar_coupling = pd.read_csv('E:/kaggle/Molecular_properties/champs-scalar-coupling/scalar_coupling_contributions.csv')

Wall time: 8.45 s


In [62]:
scalar_coupling.shape

(4658147, 8)

In [63]:
scalar_coupling.head()

,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
0,dsgdb9nsd_000001,1,0,1JHC,83.022400000000005,0.254579,1.25862,0.272010
1,dsgdb9nsd_000001,1,2,2JHH,-11.034700000000001,0.352978,2.85839,-3.433600
2,dsgdb9nsd_000001,1,3,2JHH,-11.032500000000001,0.352944,2.85852,-3.433870
3,dsgdb9nsd_000001,1,4,2JHH,-11.031900000000000,0.352934,2.85855,-3.433930
4,dsgdb9nsd_000001,2,0,1JHC,83.022199999999998,0.254585,1.25861,0.272013


In [ ]:
#One model propotype is to get all the contributions and use the contributions to get the final results